# Design of Experiments and Optimization: Pellet mill
This is the second notebook regarding DoE-RSM-Optimization workflow.
The difference is that we take the DoE table obtained earlier in the course and construct the response surface accordingly.
In the optimization part, we additionally consider an optimization problem that applies when one wants to track a specific PDI value rather than the maximum.

## Getting response surface

In [ ]:
import numpy as np
import pandas as pd
from pyscipopt import Model, quicksum

In [ ]:
# Import the DoE table
datafile = 'btgdat.xlsx'
df = pd.read_excel(datafile, sheet_name='Factorial CRD 3 factor')
print(df)

In [ ]:
x = df[['A', 'B', 'C']].to_numpy().T
y = df['PDI'].to_numpy().T

m,n = np.shape(x) # m = number of factors, n = number of runs

In [ ]:
# Construct the matrix X by its transpose.
## First, stack ones over the linear terms.
X = np.vstack((np.ones(n),x))
## Second, stack the above matrix above the quadratic terms.
for i in range(m):
    for j in range(i,m):
        X = np.vstack((X,x[i,:]*x[j,:]))
X = X.T

In [ ]:
# Do quadratic regression 
u = np.linalg.lstsq(X.T@X,X.T@y)[0]
c = u[0]
b = u[1:1+m].reshape(-1,1)
A = np.zeros((m,m))
A[np.triu_indices(3)] = u[1+m:]
A = .5*(A + A.T)
q = lambda x: x.T@A@x + b.T@x + c

In [ ]:
for i in range(n):
    print(f"Run#{i}:\t x={x[:,i]}\t  y={y[i]}\t  P={q(x[:,i])[0]}")

----

## Optimization: Maximizing PDI

In [ ]:
maxpdi = Model()
xx = [maxpdi.addVar(vtype='C', lb=-1, ub=1, name=f"x{i}") for i in range(m)]
obj = maxpdi.addVar(vtype='C', name="obj")
maxpdi.setObjective(obj, sense='maximize')
maxpdi.addCons( obj <= 
                        quicksum(A[i][j]*xx[i]*xx[j] for i in range(m) for j in range(m))
                        + quicksum(b[i][0]*xx[i] for i in range(m)) + c )
maxpdi.hideOutput()
maxpdi.optimize()
SOL = maxpdi.getBestSol()
x_max = [SOL[xx[i]] for i in range(m)]
OBJ = SOL[obj]
print(f"The maximum PDI={OBJ} is achieved at x1={x_max[0]} x2={x_max[1]} x3={x_max[2]}.")

----

## Optimization: Tracking a specific PDI value

Instead of the PDI value as high as possible, we may want a specific PDI value $p^{\ast}$ that we prefer our pellets to have.

Since we want to be as close to $p^{\ast}$ as possible, the **square distance** between $q(x)$ and $p^{\ast}$ is minimized:
$$ 
\begin{array}{cl}
\min \quad &J(x) := (q(x) - p^{\ast})^{2} \\
\text{s.t.}\quad & x \in [-1,1]^{m}.
\end{array}
$$

This is no longer a quadratic program, and it is potentially a **nonconvex optimization problem**.
With the structure, we aim to use the projected gradient descent method to solve this problem.
Recalling that $q(x) = x^{\top}Ax + b^{\top} + c$, we have $\nabla q(x) = 2Ax + b$ and then
$$
\nabla J(x) = 2(q(x) - p^{\ast}) \nabla q(x) = 2(q(x) - p^{\ast})(2Ax + b).
$$
The **projected gradient descent method (PGDM)** with diminishing rate is described as follows.

- **Initialization** Pick an initial point $x$ and an initial learning rate $\alpha > 0$.
- **Repeat**
   * Compute: $q(x^{0})$
   * Update learning rate: $\alpha \leftarrow \alpha/2$
   * Gradient step: $x \leftarrow x - 2\alpha(q(x) - p^{\ast})(2Ax + b)$
   * Projection step:
       * For $i = 1,\dots,m$;
           - If $x_{i} \leq -1$, $x_{i} \leftarrow -1$;
           - If $x_{i} \geq 1$, $x_{i} \leftarrow 1$;
- **Until** A stopping criterion is satisfied.
- **Return** $x$

In [ ]:
# Check the eigenvalues of A for convexity check.
np.linalg.eig(A)[0] >= 0

In [ ]:
p_star = 94.0
J = lambda x: (q(x) - p_star)**2

XX = np.array([1,1,1.]).reshape((-1,1))
alpha = 0.001

for k in range(20):
    qq = q(XX)
    alpha /= 2
    XX -= 2*alpha*(qq - p_star)*(2*A@XX + b)
    # print(f"Step{k}\t x = {XX}")
    for i in range(m):
        if XX[i] <= -1:
            XX[i] = -1
        elif XX[i] >= 1:
            XX[i] = 1
    # print(f"After proj, x = {XX}")
print(f"x = {XX.reshape((-1,))} q(x) = {q(XX)}")

### The failure of OFF-THE-SHELF solvers
Eventhough PySCIPOpt is advertised to solve nonlinear (nonconvex, actually) problems, it has its limitation outside the quadratic realm.

The following shows that PySCIPOpt fails to track the value of $p^{\ast}$.

In [ ]:
m2 = Model()
t = m2.addVar(vtype='C', name='t')
xx = [m2.addVar(vtype='C', lb=-1, ub=1, name=f'x{i}') for i in range(m)]
Obj = m2.addVar(vtype='C', name="obj")
m2.setObjective(obj, sense='minimize')
m2.addCons( (t - p_star)**2 <= Obj )
m2.addCons( t == 
                  quicksum(A[i][j]*xx[i]*xx[j] for i in range(m) for j in range(m))
                  + quicksum(b[i][0]*xx[i] for i in range(m)) + c )
m2.hideOutput()
m2.optimize()

In [ ]:
SOL = m2.getBestSol()
print(SOL)

Note that the solution from the PySCIPOpt does not make sense, as the value of `Obj` should at most be $(t-p^{\ast})^{2}$.

----